# What do we have to do?

- Data acquisition
- Preparation
- Cleaning and aggregation
- Exploratory data analyis


# Description

xx
xx
xx

***

# Data
Main datasets:
- **movies_metadata.csv** - Movies dataset
- **credits_cleaned01.csv** - List of people per movie (directors, actors, etc)

***


In [64]:
import pandas as pd

## Movies

In [65]:
import pandas as pd
movies = pd.read_csv('data/movies_metadata.csv',dtype={'popularity': 'float'})
print('Columns:')
print(',  '.join(movies))

Columns:
adult,  belongs_to_collection,  budget,  genres,  homepage,  id,  imdb_id,  original_language,  original_title,  overview,  popularity,  poster_path,  production_companies,  production_countries,  release_date,  revenue,  runtime,  spoken_languages,  status,  tagline,  title,  video,  vote_average,  vote_count


In [66]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415


***
## CREDITS
This dataframe contains 3 columns:
- **cast**: Json string with the information about the actor/acrtrees
- **crew**: Json string with the information about Directors and other members of the team
- **id**: Movie identifier

***

First we import json to to be able to deal with json strings:

In [46]:
import json
from os.path import exists

**credits_cleaned01** is big in size, the first action we are doing here is removing the column **crew** since we are not going to use it and then we save the dataset as a new file (which size is almost a half of the original dataset)

## REDUCING CREDITS FILE SIZE

**RUN ALL THIS IF YOU HAVE THE credits_cleaned01.csv FILE,**

This file is greater than 100mb, we can not upload it into github, skip this part and run the section where we import the file **credits_cleaned03.csv** down below.

In [477]:
file_exists = exists("data/credits_cleaned01.csv")
if(file_exists):
    credits = pd.read_csv('data/credits_cleaned01.csv')
    credits.drop(columns=['crew'], inplace=True, axis=1)
    credits.to_csv('data/credits_cleaned02.csv', index=False)
    display(list(credits.columns))

['Unnamed: 0', 'cast', 'id']

Now we read again the new file  and displays its size and columns where we can see there are only two 

In [493]:
file_exists = exists("data/credits_cleaned02.csv")
if(file_exists):
    credits = pd.read_csv('data/credits_cleaned02.csv',index_col=0)
    display(len(credits))
    display(list(credits.columns))

45399

['cast', 'id']

Each row contains a large json with information we don't wan't want to use, this is how each **cast** looks like

In [494]:
if(file_exists):
    display(credits.iloc[0]['cast'])

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

Since the format of the json is not compatible with **json.loads()** we need to change the structure in order to have a dict of people for each movie:
- Loop through each row on the dataset
- Change quotes to dummy values
- Remove unnecesary quotes or backslash (if any)
- Remove unnecesary indexes from the dict of people
- Update the dataset with the reduced json file

In [9]:
file_exists = exists("data/credits_cleaned02.csv")
if(file_exists):
    credits = pd.read_csv('data/credits_cleaned02.csv',index_col=0)
    credits.reset_index(drop=True, inplace=True)

errors = []
if(file_exists):
    for i, credit in credits.iterrows():
        if(i >= 0):
            try:
                cast = credit.cast
                cast = cast.replace("{'", "{##")
                cast = cast.replace('{"', "{##")

                cast = cast.replace("': ", "##: ")
                cast = cast.replace('": ', "##: ")

                cast = cast.replace(", '", ", ##")
                cast = cast.replace(', "', ", ##")

                cast = cast.replace("'}", "##}")
                cast = cast.replace('"}', "##}")

                cast = cast.replace(": '", ": ##")
                cast = cast.replace("', ", "##, ")
                cast = cast.replace(': "', ': ##')
                cast = cast.replace('", ', '##, ')
                cast = cast.replace('"}', '##}')
                cast = cast.replace("\\", '')

                cast = cast.replace('"', "")
                cast = cast.replace("'", "")

                cast = cast.replace('##', '"')
                cast = cast.replace(": None", ": null")
                if(credit.cast == "[]" or credit.cast == ""):
                    errors.append(i)
                else:
                    arrayOfPeople = json.loads(cast)
                    for dictOfPeople in arrayOfPeople:
                        dictOfPeople.pop("cast_id")
                        dictOfPeople.pop("credit_id")
                        dictOfPeople.pop("gender")
                        dictOfPeople.pop("order")
                        dictOfPeople.pop("profile_path")
                        dictOfPeople.pop("id")
                        dictOfPeople['character'].replace('"',"´")
                    credits.iloc[i,0] = json.dumps(arrayOfPeople)
            except Exception as e:
                print(i)
                errors.append(i)

NameError: name 'exists' is not defined

In [556]:
#These are out of range elements:
#print(errors)

In [557]:
len(credits)

45399

In [558]:
#If we want to eliminate errors:
for i in errors:
    try:
        credits.drop(credits.index[i],inplace=True)
    except Exception as e:
        #print(e)
        pass
credits.reset_index(drop=True, inplace=True)

In [559]:
len(credits)

43182

In [560]:
credits.head(3)

,cast,id
0,"[{""character"": ""Woody (voice)"", ""name"": ""Tom H...",862
1,"[{""character"": ""Alan Parrish"", ""name"": ""Robin ...",8844
2,"[{""character"": ""Max Goldman"", ""name"": ""Walter ...",15602


Now we store this dataframe as a new file, with these steps we have reduced the original file from 190mb to 37mb which is better because we will deal with a smaller file

In [561]:
credits.to_csv('data/credits_cleaned03.csv')

## USING REDUCED CREDITS FILE 

In [4]:
credits = pd.read_csv('data/credits_cleaned03.csv', index_col=0)
credits.head(2)

,cast,id
0,"[{""character"": ""Woody (voice)"", ""name"": ""Tom H...",862
1,"[{""character"": ""Alan Parrish"", ""name"": ""Robin ...",8844


Let's get a dataset of performers counting their participation on movies:

In [5]:
actArray = []
movArray = []
for i, credit in credits.iterrows():
    try:
        credit.cast = credit.cast.replace("'", '"')
        arrayOfPeople = json.loads(credit.cast)
        for dictOfPeople in arrayOfPeople:
            actArray.append(dictOfPeople["name"])
            movArray.append(credit.id)
    except Exception as e:  
        print(i,credit.cast)
        break

    
myDict = {'act': actArray, 'mov': movArray}
acts = pd.DataFrame(data=myDict)

(acts['act'].value_counts())

Bess Flowers            230
Christopher Lee         141
Samuel L. Jackson       120
John Wayne              120
Donald Sutherland       107
                       ... 
Joanne                    1
Johnny Chung-Jen Lin      1
Ma Nien-Hsien             1
Wei-min Ying              1
Aleksandr Chabrov         1
Name: act, Length: 196194, dtype: int64

## USING MOVIES FILE

Let's analyze the column belongs_to_collection:

In [47]:
print(movies.loc[:,'belongs_to_collection'][0])
print(movies.loc[:,'belongs_to_collection'][9])

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
{'id': 645, 'name': 'James Bond Collection', 'poster_path': '/HORpg5CSkmeQlAolx3bKMrKgfi.jpg', 'backdrop_path': '/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg'}


The column has some values that we are not going to use: poster_path, backdrop_path and id.

In [48]:
errors = []
for i, movie in movies.iterrows():
    if(i >= 0):
        try:
            collection = movie.belongs_to_collection

            collection = collection.replace("{'", "{##")
            collection = collection.replace('{"', "{##")

            collection = collection.replace("': ", "##: ")
            collection = collection.replace('": ', "##: ")

            collection = collection.replace(", '", ", ##")
            collection = collection.replace(', "', ", ##")

            collection = collection.replace("'}", "##}")
            collection = collection.replace('"}', "##}")

            collection = collection.replace(": '", ": ##")
            collection = collection.replace("', ", "##, ")
            collection = collection.replace(': "', ': ##')
            collection = collection.replace('", ', '##, ')
            collection = collection.replace('"}', '##}')
            collection = collection.replace("\\", '')

            collection = collection.replace('"', "")
            collection = collection.replace("'", "")

            collection = collection.replace('##', '"')
            collection = collection.replace(": None", ": null")

            if(movie.belongs_to_collection == "[]" or movie.belongs_to_collection == ""):
                errors.append(i)
            else:
                collectionDict = json.loads(collection)
                #Removing the columns poster_path, backdrop_path, and id
                collectionDict.pop("poster_path")
                collectionDict.pop("backdrop_path")
                collectionDict.pop("id")
                movies.loc[i,'belongs_to_collection'] = json.dumps(collectionDict)
                #movies.loc[i,'belongs_to_collection'] = str(collectionDict['name'])
        except Exception as e:
            #print(e)
            errors.append(i)

In [49]:
movies.loc[:,'belongs_to_collection'][0]

'{"name": "Toy Story Collection"}'

Let's analyze the column genres:

In [50]:
print(movies.loc[:,'genres'][0])
print(movies.loc[:,'genres'][9])

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]


The column has some values that we are not going to use: id

In [51]:
errors = []
for i, movie in movies.iterrows():
    if(i >= 0):
        try:
            genres = movie.genres

            genres = genres.replace("{'", "{##")
            genres = genres.replace('{"', "{##")

            genres = genres.replace("': ", "##: ")
            genres = genres.replace('": ', "##: ")

            genres = genres.replace(", '", ", ##")
            genres = genres.replace(', "', ", ##")

            genres = genres.replace("'}", "##}")
            genres = genres.replace('"}', "##}")

            genres = genres.replace(": '", ": ##")
            genres = genres.replace("', ", "##, ")
            genres = genres.replace(': "', ': ##')
            genres = genres.replace('", ', '##, ')
            genres = genres.replace('"}', '##}')
            genres = genres.replace("\\", '')

            genres = genres.replace('"', "")
            genres = genres.replace("'", "")

            genres = genres.replace('##', '"')
            genres = genres.replace(": None", ": null")

            if(movie.genres == "[]" or movie.genres == ""):
                errors.append(i)
            else:
                genresDict = json.loads(genres)
                for genre in genresDict:
                    #Removing the column id
                    genre.pop("id")
                    #genresList.append(genre['name'])
                movies.loc[i,'genres'] = json.dumps(genresDict)
                #movies.loc[i,'genres'] = str(genresList)
        except Exception as e:
            #print(e)
            errors.append(i)

In [52]:
print(movies.loc[:,'genres'][0])

[{"name": "Animation"}, {"name": "Comedy"}, {"name": "Family"}]


Let's analyze the column production_companies:

In [53]:
movies.loc[:,'production_companies'][1]

"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"

The column has some values that we are not going to use: id

In [54]:
errors = []
for i, movie in movies.iterrows():
    if(i >= 0):
        try:
            prod_companies = movie.production_companies

            prod_companies = prod_companies.replace("{'", "{##")
            prod_companies = prod_companies.replace('{"', "{##")

            prod_companies = prod_companies.replace("': ", "##: ")
            prod_companies = prod_companies.replace('": ', "##: ")

            prod_companies = prod_companies.replace(", '", ", ##")
            prod_companies = prod_companies.replace(', "', ", ##")

            prod_companies = prod_companies.replace("'}", "##}")
            prod_companies = prod_companies.replace('"}', "##}")

            prod_companies = prod_companies.replace(": '", ": ##")
            prod_companies = prod_companies.replace("', ", "##, ")
            prod_companies = prod_companies.replace(': "', ': ##')
            prod_companies = prod_companies.replace('", ', '##, ')
            prod_companies = prod_companies.replace('"}', '##}')
            prod_companies = prod_companies.replace("\\", '')

            prod_companies = prod_companies.replace('"', "")
            prod_companies = prod_companies.replace("'", "")

            prod_companies = prod_companies.replace('##', '"')
            prod_companies = prod_companies.replace(": None", ": null")

            if(movie.production_companies == "[]" or movie.production_companies == ""):
                errors.append(i)
            else:
                prodcompDict = json.loads(prod_companies)
                for prodcomp in prodcompDict:
                    #Removing the column id
                    prodcomp.pop("id")
                movies.loc[i,'production_companies'] = json.dumps(prodcompDict)
        except Exception as e:
            #print(e)
            errors.append(i)

In [55]:
movies.loc[:,'production_companies'][1]

'[{"name": "TriStar Pictures"}, {"name": "Teitler Film"}, {"name": "Interscope Communications"}]'

Exporting the changes to a new file:

In [56]:
movies.to_csv('data/movies_cleaned01.csv', index=False)

Reading the cleaned file:

In [67]:
movies = pd.read_csv('data/movies_cleaned01.csv', dtype={'popularity': 'float'})
print('Columns:')
print(',  '.join(movies2))

Columns:
adult,  belongs_to_collection,  budget,  genres,  homepage,  id,  imdb_id,  original_language,  original_title,  overview,  popularity,  poster_path,  production_companies,  production_countries,  release_date,  revenue,  runtime,  spoken_languages,  status,  tagline,  title,  video,  vote_average,  vote_count


In [88]:
movies.loc[:,'production_companies'][1]

'[{"name": "TriStar Pictures"}, {"name": "Teitler Film"}, {"name": "Interscope Communications"}]'

In [91]:
movies.popularity

0        21.946943
1        17.015539
2        11.712900
3         3.859495
4         8.387519
           ...    
45458     0.072051
45459     0.178241
45460     0.903007
45461     0.003503
45462     0.163015
Name: popularity, Length: 45463, dtype: float64

In [97]:
movies.id.sort_values()

4342          2
12947         3
17            5
474           6
256          11
          ...  
45075    465044
45270    467731
21890    468343
45395    468707
20188    469172
Name: id, Length: 45463, dtype: int64

In [95]:
movies[movies.id=='6269']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


## USING RATINGS FILE

In [98]:
ratings = pd.read_csv('data/ratings_small.csv', index_col=0)
ratings.head()

,movieId,rating,timestamp
userId,,,
1,31,2.5,1260759144
1,1029,3.0,1260759179
1,1061,3.0,1260759182
1,1129,2.0,1260759185
1,1172,4.0,1260759205


In [103]:
type(ratings.movieId)

pandas.core.series.Series